# Deploy Watson ML (PMML)

Deploys a PMML model to IBM Watson Machine Learning (WML)

In [1]:
!pip install ibm-watson-machine-learning==1.0.45
# PLEASE RESTART YOUR KERNAL AFTER THIS LINE HAS BEEN EXECUTED

In [2]:
from ibm_watson_machine_learning import APIClient
import logging
import os
import re
import sys

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [3]:
# IBM Cloud API Key https://cloud.ibm.com/iam/apikeys
api_key = os.environ.get('api_key', 'miutmhePHqus2AXdHlzKqJjkE2rXFckAHY89rux1JJO-')

# Machine Learning Model Deployment Space https://dataplatform.cloud.ibm.com/ml-runtime/spaces
space = os.environ.get('space', '9786c1e5-daf5-49a1-9893-6c14e0eb307c')

# IBM Cloud Region (e.g. us-south)
location = os.environ.get('location', 'us-south')

# temporary directory for data
data_dir = os.environ.get('data_dir',
                          '../../data/')

In [4]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [5]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + 'us-south' + '.ml.cloud.ibm.com'
}

In [6]:
client = APIClient(wml_credentials)

In [7]:
o = client.software_specifications.get_uid_by_name('pmml-3.0_4.3')
software_spec_uid = o
client.set.default_space(space)

2022-09-28 04:22:46,251 - ibm_watson_machine_learning.wml_client_error - WARNING - Space with id '9786c1e5daf549a198936c14e0eb307c' does not exist


WMLClientError: Space with id '9786c1e5daf549a198936c14e0eb307c' does not exist

In [ ]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: 'test_pmml2',
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model=data_dir + 'model.xml',
    meta_props=model_meta_props,
)

model_uid = client.repository.get_model_uid(published_model)